# pems-bay.h5

In [2]:
import h5py
import pickle
import folium
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
with h5py.File('data/PEMS-BAY/pems-bay.h5', 'r') as file:

    axis0 = file['speed']['axis0'][:]               # Идентификаторы датчиков
    block0_items = file['speed']['block0_items'][:] # Идентификаторы датчиков
    axis1 = file['speed']['axis1'][:]               # Метки времени
    timestamps = pd.to_datetime(axis1)              # Преобразование меток времени в формат datetime
    speed_data = file['speed']['block0_values'][:]  # Данные замеров скорости

perms_bay = pd.DataFrame(speed_data, index=timestamps, columns=axis0)

In [4]:
perms_bay.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52116 entries, 2017-01-01 00:00:00 to 2017-06-30 23:55:00
Columns: 325 entries, 400001 to 414694
dtypes: float64(325)
memory usage: 129.6 MB


In [5]:
# Открытие .pkl файла
with open('data/PEMS-BAY/adj_mx_bay.pkl', 'rb') as file:
    data = pickle.load(file, encoding='bytes')

In [6]:
node_ids = [x.decode('utf-8') for x in data[0]]                     # Получаем список id узлов из data[0]
adj_matrix = data[2]                                                # Получаем матрицу смежности из data[2]
adj_df = pd.DataFrame(adj_matrix, index=node_ids, columns=node_ids) # Создание DataFrame с использованием id узлов как индексов и названий колонок

In [7]:
distances_df = pd.read_csv('data/PEMS-BAY/distances_bay_2017.csv', header=None)
locations_df = pd.read_csv('data/PEMS-BAY/graph_sensor_locations_bay.csv', header=None)

In [8]:
distances_df.columns = ['from', 'to', 'distance']

# Model


In [9]:
import numpy as np
import pandas as pd
import torch
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh

class TrafficGraphData:
    def __init__(self, adj_matrix, time_series, input_len, pred_len, train_ratio=0.7, val_ratio=0.15):
        """
        Параметры:
        - adj_matrix: pd.DataFrame — матрица смежности (размер N x N).
        - time_series: pd.DataFrame — временные ряды (размер T x N).
        - input_len: int — длина входного окна.
        - pred_len: int — длина окна предсказания.
        - train_ratio: float — доля данных для тренировки.
        - val_ratio: float — доля данных для валидации.
        """
        self.adj_matrix = adj_matrix.values.astype(np.float32)
        self.time_series = time_series.values.astype(np.float32)
        self.input_len = input_len
        self.pred_len = pred_len
        
        # Нормализованная матрица Лапласа
        self.norm_laplas = self._compute_norm_laplacian(self.adj_matrix)
        
        # Разделение данных
        self.train_data, self.val_data, self.test_data = self._split_data(train_ratio, val_ratio)
    
    def _compute_norm_laplacian(self, adj_matrix):
        """
        Вычисляет нормализованную матрицу Лапласа для графа.
        """
        N = adj_matrix.shape[0]
        adj_matrix += np.eye(N)  # Добавление self-loops
        degree_matrix = np.diag(adj_matrix.sum(axis=1))
        degree_inv_sqrt = np.linalg.inv(np.sqrt(degree_matrix))
        laplacian = degree_inv_sqrt @ adj_matrix @ degree_inv_sqrt
        return torch.tensor(laplacian, dtype=torch.float32)
    
    def _create_sliding_window(self, data):
        """
        Создаёт скользящие окна для временных рядов.
        Возвращает:
        - X: np.array — входные данные (размер [кол-во окон, input_len, N]).
        - Y: np.array — целевые значения (размер [кол-во окон, pred_len, N]).
        """
        X, Y = [], []
        for i in range(len(data) - self.input_len - self.pred_len + 1):
            X.append(data[i : i + self.input_len])
            Y.append(data[i + self.input_len : i + self.input_len + self.pred_len])
        return np.array(X), np.array(Y)
    
    def _split_data(self, train_ratio, val_ratio):
        """
        Делит временные данные на тренировочные, валидационные и тестовые наборы.
        """
        X, Y = self._create_sliding_window(self.time_series)
        total_len = len(X)
        
        train_size = int(train_ratio * total_len)
        val_size = int(val_ratio * total_len)
        
        train_data = (X[:train_size], Y[:train_size])
        val_data = (X[train_size:train_size + val_size], Y[train_size:train_size + val_size])
        test_data = (X[train_size + val_size:], Y[train_size + val_size:])
        
        return train_data, val_data, test_data

    def get_train_data(self):
        return self.train_data
    
    def get_val_data(self):
        return self.val_data
    
    def get_test_data(self):
        return self.test_data

In [11]:
# Инициализация класса
input_len = 12       # Входное окно: 1 час (если шаг 5 минут)
pred_len = 3         # Окно предсказания: 15 минут
graph_data = TrafficGraphData(adj_df, perms_bay, input_len, pred_len)

# Получение данных
train_data = graph_data.get_train_data()
val_data = graph_data.get_val_data()
test_data = graph_data.get_test_data()

# Матрица Лапласа
laplacian_matrix = graph_data.norm_laplas

In [10]:
import torch
from torch.utils.data import DataLoader, TensorDataset

class TrafficGraphDataset:
    def __init__(self, graph_data):
        """
        Класс для создания датасета для GCN на основе подготовленных данных.
        Параметры:
        - graph_data: TrafficGraphData — объект с данными графа и временными рядами.
        """
        self.train_data = graph_data.get_train_data()
        self.val_data = graph_data.get_val_data()
        self.test_data = graph_data.get_test_data()

    def create_dataloader(self, data, batch_size):
        """
        Создаёт DataLoader для удобной работы с данными.
        """
        X, Y = data
        X_tensor = torch.tensor(X, dtype=torch.float32)
        Y_tensor = torch.tensor(Y, dtype=torch.float32)
        dataset = TensorDataset(X_tensor, Y_tensor)
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)

    def get_dataloaders(self, batch_size=32):
        """
        Возвращает DataLoader'ы для тренировки, валидации и теста.
        """
        train_loader = self.create_dataloader(self.train_data, batch_size)
        val_loader = self.create_dataloader(self.val_data, batch_size)
        test_loader = self.create_dataloader(self.test_data, batch_size)
        return train_loader, val_loader, test_loader

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class STGCN(nn.Module):
    def __init__(self, laplacian, input_len, num_nodes, num_features, output_len, hidden_dim=64):
        """
        Параметры:
        - laplacian: torch.Tensor — нормализованная матрица Лапласа (\( \tilde{A} \)).
        - input_len: int — длина входного временного окна.
        - num_nodes: int — количество узлов (N).
        - num_features: int — число признаков на узел.
        - output_len: int — длина окна предсказания.
        - hidden_dim: int — размер скрытого слоя.
        """
        super(STGCN, self).__init__()
        
        self.laplacian = laplacian
        self.input_len = input_len
        self.num_nodes = num_nodes
        self.num_features = num_features
        self.output_len = output_len
        self.hidden_dim = hidden_dim

        # GCN слой
        self.gcn = nn.Linear(num_features, hidden_dim)

        # Временная свёртка
        self.temporal_conv = nn.Conv2d(
            in_channels=hidden_dim,
            out_channels=hidden_dim,
            kernel_size=(3, 1),  # Свёртка по времени
            padding=(1, 0)  # Чтобы сохранить размер
        )
        
        # Полносвязный слой для предсказания
        self.fc = nn.Linear(hidden_dim, output_len)

    def forward(self, x):
        """
        x: torch.Tensor — входной тензор [batch_size, input_len, num_nodes, num_features].
        Возвращает: Предсказания [batch_size, output_len, num_nodes].
        """
        if x.dim() == 3:  # Если [batch_size, input_len, num_nodes]
            x = x.unsqueeze(-1)  # Добавляем размер признаков [batch_size, input_len, num_nodes, num_features]
        batch_size = x.shape[0]
        

        # GCN операция: учёт связей между узлами
        x = x.permute(0, 2, 1, 3)  # [batch_size, num_nodes, input_len, num_features]
        x = x.reshape(-1, self.num_features)  # [batch_size * num_nodes, num_features]
        x = F.relu(torch.matmul(self.laplacian, self.gcn(x)))  # [batch_size * num_nodes, hidden_dim]
        x = x.view(batch_size, self.num_nodes, self.input_len, self.hidden_dim)
        x = x.permute(0, 3, 2, 1)  # [batch_size, hidden_dim, input_len, num_nodes]

        # Временная свёртка
        x = F.relu(self.temporal_conv(x))  # [batch_size, hidden_dim, input_len, num_nodes]

        # Сжатие по времени и выход
        x = x.mean(dim=2)  # Убираем временную ось [batch_size, hidden_dim, num_nodes]
        x = self.fc(x)  # [batch_size, num_nodes, output_len]
        x = x.permute(0, 2, 1)  # [batch_size, output_len, num_nodes]

        return x

In [21]:
# Параметры модели
input_len = 12       # Входное окно: 12 шагов
pred_len = 3         # Выходное окно: 3 шага
num_nodes = adj_df.shape[0]
num_features = 1     # Одно значение на узел (например, поток)
hidden_dim = 64

# Инициализация модели
laplacian = graph_data.norm_laplas
model = STGCN(laplacian, input_len, num_nodes, num_features, pred_len, hidden_dim)

# Пример данных
X_train, Y_train = graph_data.get_train_data()
X_train_tensor = torch.tensor(X_train[:1, :, :], dtype=torch.float32)  # [batch_size, input_len, num_nodes, num_features]
Y_train_tensor = torch.tensor(Y_train[:1, :, :], dtype=torch.float32)  # [batch_size, pred_len, num_nodes]

In [25]:
f"{X_train_tensor.shape = }"

'X_train_tensor.shape = torch.Size([1, 12, 325])'

In [23]:
# Прямой проход
predictions = model(X_train_tensor)
print(predictions.shape)  # Ожидаем: [batch_size, pred_len, num_nodes]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (325x325 and 3900x64)

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class STGCN(nn.Module):
    def __init__(self, laplacian, input_len, num_nodes, num_features, output_len, hidden_dim=64):
        super(STGCN, self).__init__()
        
        self.laplacian = laplacian  # [num_nodes, num_nodes]
        self.input_len = input_len
        self.num_nodes = num_nodes
        self.num_features = num_features
        self.output_len = output_len
        self.hidden_dim = hidden_dim

        # GCN layer
        self.gcn = nn.Linear(num_features, hidden_dim)

        # Temporal convolution
        self.temporal_conv = nn.Conv2d(
            in_channels=hidden_dim,
            out_channels=hidden_dim,
            kernel_size=(3, 1),  # Convolution along time
            padding=(1, 0)  # Maintain size
        )
        
        # Fully connected layer for prediction
        self.fc = nn.Linear(hidden_dim, output_len)

    def forward(self, x):
        """
        x: torch.Tensor — input tensor [batch_size, input_len, num_nodes, num_features].
        Returns: Predictions [batch_size, output_len, num_nodes].
        """
        if x.dim() == 3:
            x = x.unsqueeze(-1)  # Add feature dimension [batch_size, input_len, num_nodes, num_features]
        print(f"Step 1 - Input shape after adding feature dim: {x.shape}")

        batch_size = x.shape[0]

        # GCN operation for each time step
        x = x.permute(0, 2, 1, 3)  # [batch_size, num_nodes, input_len, num_features]
        print(f"Step 2 - Shape after permute: {x.shape}")
        x = x.reshape(batch_size * self.num_nodes, self.input_len, self.num_features)  # [batch_size * num_nodes, input_len, num_features]
        print(f"Step 3 - Shape after reshaping for GCN: {x.shape}")
        
        x = F.relu(self.gcn(x))  # Apply GCN layer: [batch_size * num_nodes, input_len, hidden_dim]
        print(f"Step 4 - Shape after GCN layer: {x.shape}")
        
        # Apply Laplacian to each time step
        x = x.view(batch_size, self.num_nodes, self.input_len, self.hidden_dim)  # [batch_size, num_nodes, input_len, hidden_dim]
        x = torch.einsum('ij,bjtl->bitl', self.laplacian, x)  # Graph convolution using Laplacian: [batch_size, num_nodes, input_len, hidden_dim]
        print(f"Step 5 - Shape after Laplacian multiplication: {x.shape}")

        # Temporal convolution
        x = x.permute(0, 3, 2, 1)  # [batch_size, hidden_dim, input_len, num_nodes]
        x = F.relu(self.temporal_conv(x))  # [batch_size, hidden_dim, input_len, num_nodes]
        print(f"Step 6 - Shape after temporal convolution: {x.shape}")

        # Reduce along the time axis
        x = x.mean(dim=2)  # Remove time axis [batch_size, hidden_dim, num_nodes]
        print(f"Step 7 - Shape after reducing along time axis: {x.shape}")
        
        x = x.permute(0, 2, 1)  # [batch_size, num_nodes, hidden_dim]
        x = self.fc(x)  # [batch_size, num_nodes, output_len]
        x = x.permute(0, 2, 1)  # [batch_size, output_len, num_nodes]
        print(f"Step 8 - Final prediction shape: {x.shape}")

        return x

# Тестируем модель
laplacian = torch.eye(325)  # Identity matrix for testing
input_len = 12
num_nodes = 325
num_features = 1
output_len = 3
hidden_dim = 64

# Данные
X_train_tensor = torch.randn(1, 12, 325)  # Simulated input data

# Инициализируем модель
model = STGCN(laplacian, input_len, num_nodes, num_features, output_len, hidden_dim)

# Forward pass
predictions = model(X_train_tensor)
print(f"Predictions shape: {predictions.shape}")

Step 1 - Input shape after adding feature dim: torch.Size([1, 12, 325, 1])
Step 2 - Shape after permute: torch.Size([1, 325, 12, 1])
Step 3 - Shape after reshaping for GCN: torch.Size([325, 12, 1])
Step 4 - Shape after GCN layer: torch.Size([325, 12, 64])
Step 5 - Shape after Laplacian multiplication: torch.Size([1, 325, 12, 64])
Step 6 - Shape after temporal convolution: torch.Size([1, 64, 12, 325])
Step 7 - Shape after reducing along time axis: torch.Size([1, 64, 325])
Step 8 - Final prediction shape: torch.Size([1, 3, 325])
Predictions shape: torch.Size([1, 3, 325])


In [32]:
pd.DataFrame(predictions[0].detach().numpy())

,0,1,2,3,4,5,6,7,8,9,...,315,316,317,318,319,320,321,322,323,324
0,0.022213,0.009949,0.019647,0.000767,0.015424,0.023195,0.041739,-0.014152,0.016560,0.025389,...,0.007506,0.023157,0.003008,0.016188,0.001956,0.009272,0.006271,-0.005405,-0.001155,0.031597
1,0.065086,0.036740,0.044935,0.055187,0.058353,0.064074,0.050105,0.033043,0.054188,0.063349,...,0.051942,0.055016,0.063394,0.056493,0.057803,0.051921,0.057320,0.053621,0.043851,0.041138
2,0.207885,0.228600,0.217451,0.216187,0.212961,0.226008,0.221211,0.210272,0.226671,0.231599,...,0.218479,0.237429,0.203693,0.216754,0.220643,0.213421,0.209715,0.213363,0.243100,0.247521


In [64]:
from torch_geometric_temporal.dataset import METRLADatasetLoader, PemsBayDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

# Загрузка данных
loader = PemsBayDatasetLoader()
dataset = loader.get_dataset(12, 3)

# Разделение на train и test
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

In [67]:
for snapshot in dataset:
    print(snapshot.x.shape, snapshot.y.shape, snapshot.edge_index.shape)
    break

torch.Size([325, 2, 12]) torch.Size([325, 2, 3]) torch.Size([2, 2694])


In [60]:
from torch_geometric_temporal.nn.recurrent import GConvGRU
from torch_geometric_temporal.nn.attention.gman import SpatialAttention, TemporalAttention
import torch.nn.functional as F
import torch

class TrafficPredictionModel(torch.nn.Module):
    def __init__(self, num_nodes, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.gconvgru = GConvGRU(input_dim, hidden_dim, num_nodes)
        self.spatial_attention = SpatialAttention(num_nodes, hidden_dim, bn_decay=0.9)
        self.temporal_attention = TemporalAttention(d=hidden_dim, K=4, bn_decay=0.9, mask=False)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index, edge_weight):
        # Рекуррентный слой
        h = self.gconvgru(x, edge_index, edge_weight)
        
        # Пространственное внимание
        h = self.spatial_attention(h)
        
        # Временное внимание
        h = self.temporal_attention(h)
        
        # Полносвязный слой
        out = self.fc(h)
        return out

In [88]:
import torch
import numpy as np
from torch_geometric_temporal.dataset import PemsBayDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split
import torch_geometric.transforms as T

# # Load the dataset
# loader = PemsBayDatasetLoader()
# dataset = loader.get_dataset(12, 3)  # 12 timesteps input, 3 timesteps output
# train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

# Prepare the model
class TrafficPredictionModel(torch.nn.Module):
    def __init__(self, num_nodes, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.gconvgru = GConvGRU(input_dim, hidden_dim, num_nodes)
        self.spatial_attention = SpatialAttention(num_nodes, hidden_dim, bn_decay=0.9)
        self.temporal_attention = TemporalAttention(d=hidden_dim, K=4, bn_decay=0.9, mask=False)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index, edge_weight):
        h = self.gconvgru(x, edge_index, edge_weight)
        h = self.spatial_attention(h)
        h = self.temporal_attention(h)
        out = self.fc(h)
        return out

# Data Preparation Function
def prepare_data_for_model(dataset):
    # Extract key components
    x = torch.tensor(dataset.features, dtype=torch.float)  # Node features
    edge_index = torch.tensor(dataset.edge_index, dtype=torch.long)  # Graph connectivity
    edge_weight = torch.tensor(dataset.edge_weight, dtype=torch.float)  # Edge weights
    y = torch.tensor(dataset.targets, dtype=torch.float)  # Target values

    return x, edge_index, edge_weight, y

# Prepare training and test data
x_train, edge_index_train, edge_weight_train, y_train = prepare_data_for_model(train_dataset)
x_test, edge_index_test, edge_weight_test, y_test = prepare_data_for_model(test_dataset)

# Instantiate the model
num_nodes = x_train.shape[1]  # Number of nodes
input_dim = x_train.shape[2]  # Input feature dimension
hidden_dim = 64  # You can adjust this
output_dim = y_train.shape[2]  # Output feature dimension

model = TrafficPredictionModel(num_nodes, input_dim, hidden_dim, output_dim)

# Training loop (example)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(100):  # Number of epochs
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output = model(x_train, edge_index_train, edge_weight_train)
    
    # Compute loss
    loss = criterion(output, y_train)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
model.eval()
with torch.no_grad():
    test_output = model(x_test, edge_index_test, edge_weight_test)
    test_loss = criterion(test_output, y_test)
    print(f'Test Loss: {test_loss.item()}')

RuntimeError: index 2 is out of bounds for dimension 0 with size 2

In [73]:
# Параметры модели
num_nodes = 325  # Количество узлов в графе
input_dim = 2    # Размерность входных признаков
hidden_dim = 64  # Размерность скрытого состояния
output_dim = 3   # Размерность выхода

epochs = 10
learning_rate = 0.001

In [74]:
def process_batch(batch):
    # Преобразование формата входных данных [N, F, T] -> [N, T, F]
    x = batch.x.permute(0, 2, 1)
    
    # Получение edge_index и edge_weight
    edge_index = batch.edge_index
    edge_weight = batch.edge_attr
    
    return x, edge_index, edge_weight

In [75]:
from torch.nn import MSELoss
from torch.optim import Adam

model = TrafficPredictionModel(num_nodes, input_dim, hidden_dim, output_dim)
optimizer = Adam(model.parameters(), lr=learning_rate)
loss_fn = MSELoss()

In [84]:
len(list(train_dataset))

41672

In [85]:
import torch
from tqdm import tqdm

# Включаем оптимизацию памяти
torch.cuda.empty_cache()

len_train_dataset = len(list(train_dataset))

# Функция для обработки батча с градиентным накоплением
def train_with_gradient_accumulation(model, train_dataset, optimizer, epochs, 
                                   accumulation_steps=4):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        optimizer.zero_grad()
        
        # Создаем tqdm с описанием прогресса
        pbar = tqdm(enumerate(train_dataset), 
                   total=len_train_dataset,
                   desc=f'Эпоха {epoch+1}/{epochs}',
                   leave=True)
        
        for idx, batch in pbar:
            # Перемещаем данные на устройство
            x, edge_index, edge_weight = process_batch(batch)
            x = x.to(device)
            edge_index = edge_index.to(device)
            edge_weight = edge_weight.to(device)
            y = batch.y.to(device)
            
            # Прямой проход с использованием half precision
            with torch.cuda.amp.autocast():
                out = model(x, edge_index, edge_weight)
                loss = F.mse_loss(out, y)
                loss = loss / accumulation_steps
            
            # Обратное распространение
            loss.backward()
            
            # Обновление весов после накопления градиентов
            if (idx + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            # Обновление прогресс-бара
            total_loss += loss.item() * accumulation_steps
            avg_loss = total_loss / (idx + 1)
            pbar.set_postfix({'loss': f'{avg_loss:.4f}'})
            
        # Очистка памяти в конце эпохи    
        torch.cuda.empty_cache()

train_with_gradient_accumulation(model, train_dataset, optimizer, epochs, accumulation_steps=4)

Эпоха 1/10:   0%|          | 0/41672 [00:00<?, ?it/s]C:\Users\romab\AppData\Local\Temp\ipykernel_3760\1753902722.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
d:\IT\GitHub\ITMO_subjects\.venv\lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Эпоха 1/10:   0%|          | 0/41672 [00:01<?, ?it/s]


RuntimeError: index 12 is out of bounds for dimension 0 with size 12

In [77]:
from tqdm.notebook import tqdm
# Обучение модели
model = TrafficPredictionModel(num_nodes, input_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataset):
        optimizer.zero_grad()
        
        # Подготовка данных
        x, edge_index, edge_weight = process_batch(batch)
        y = batch.y
        
        # Прямой проход
        out = model(x, edge_index, edge_weight)
        
        # Расчет потерь
        loss = F.mse_loss(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_dataset):.4f}")

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3461120000 bytes.

In [70]:
from torch_geometric.utils import add_self_loops, get_laplacian


# Step 5: Define training loop
def train_model(model, train_dataset, optimizer, loss_fn, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for snapshot in train_dataset:
            optimizer.zero_grad()
            
            # Extract data from the snapshot
            x, edge_index, edge_weight, y = snapshot.x, snapshot.edge_index, snapshot.edge_attr, snapshot.y
            edge_index, edge_weight = add_self_loops(edge_index, edge_attr=edge_weight, num_nodes=num_nodes)

            # Forward pass
            y_pred = model(x, edge_index, edge_weight)

            # Compute loss
            loss = loss_fn(y_pred, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_dataset):.4f}")

# Step 6: Train the model
train_model(model, train_dataset, optimizer, loss_fn, epochs)

RuntimeError: index 2 is out of bounds for dimension 0 with size 2